# Projet Neo4j sur la base de données BROT

### Contexte 

Pour ce projet visant à travailler avec une base de données sous forme de graphe avec neo4j, j'ai souhaité mettre à profit cette technologie dans le domaine de l'écologie (étude des populations et des écosystèmes). 

Au cours de mes recherches, j'ai découvert une base de données recensant un grand nombre d'espèce méditérranénnes ainsi que leur traits. Les traits étant des charactéristiques phénotypiques, génétiques ou morphologiques d'un organisme (ref). Les auteurs de cette base de données sont Çağatay Tavşanoğlu & Juli G. Pausas. Ils ont référencé 2457 taxons méditérranéens et un total de 44 traits, pour obtenir au final 25 764 individus, un individu étant la mesure d'un trait pour un taxon donné (tous les traits ne sont pas recensés pour chaque taxons). 

Cette base de données a été constituée de plusieurs manières notamment grâce à des références bibliographiques et des observations expérimentales. Ainsi, pour chaque paire taxon-trait observé, donc pour chaque ligne, il y a également une source bibliographique. 

La base de données est donc composée en 4 fichiers csv : 

- BROT2_dat.csv (toutes les paires taxon-trait observé)
- BROT2_sou.csv (toutes les sources)
- BROT2_tax.csv (toutes les informations supplémentaires sur les taxons)
- BROT2_syn.csv (tous les synonymes des taxons)

J'ai également décidé de créer un fichier **traits.csv**, permettant d'avoir le nom complet d'un trait et des explications le concernant. Ces informations sont disponibles dans l'article sur lequel j'ai basé mon projet (ref). J'ai décidé de créer ce fichier afin d'avoir l'information directement dans le résultat de mes requêtes et ne pas avoir à me référer systématiquement à l'article pour analyser mes résultats.


## Problématique 
A partir de ce jeu de données, les problématiques possibles sont très nombreuses. D'après l'article ce type de base de données (sur les traits) est très prometteur pour les biologistes afin de comprendre la répartition de populations et la formation d'écosystèmes. Les sujets de réflexions étant vastes, j'ai décidé de me concentrer sur certaines questions que je me suis posé au premier abord en lisant l'article.

Dans un premier temps, comme l'auteur a identifié des traits en lien avec la résillience des plantes au incendie, j'ai décidé de me concentrer sur l'étude de ces traits. En effet, la présence du feu en méditérranée a eu un impact sur le développement des populations y compris végétales. Certaines espèces ont développé des traits traduisant une adaptation à cette pression environnementale particulière. 

Je me suis donc demandé si l'on pouvait identifier un pattern de plantes adpatées à la présence du feu en méditérranée. Comme nous allons le voir par la suite, certains traits sont des mesures directes de l'adaptation aux incendies. Mais je voulais savoir si parmis les plantes qui possèdent ces traits, nous pouvions mettre en évidence d'autres traits qui ne serait pas directement associés à l'adaptation au feu.

## Adaptation aux incendies

Afin de répondre à la première probématique posée, il nous faut définir ce que sont les taxons présentants des adaptations aux incendies. A partir de la description des traits dans l'article, j'ai pu définir un ensemble de trait définissant une adaptation aux incendies, la modalité choisie est indiquée en ().

- Fire-stimulated ﬂowering **FireStimFlower** (yes)
- Resprouting capacity after ﬁre **RespFire** (high)
- Chemical germination cues **ChemCues** (stimulation)
- Heat-stimulated germination **HeatStimGerm** (high|LMH)
- Post-ﬁre seedling emergence **SeedlEmerg** (high)
- Post-ﬁre seedling survival **SeedlSurv** (high)

Afin de connaître la description de chacun des traits choisis, nous pouvons interoger la base de données avec une première requête simple. 

In [95]:
fireTraits = ["FireStimFlower", "RespFire", "ChemCues", "HeatStimGerm", "SeedlEmerg", "SeedlSurv"]

for trait in fireTraits:
    rq = '''match(t:Trait {name : $trait}) return t.name as name, t.explanation as explanation'''
    res = graph.run(rq, trait=trait).to_data_frame()
    print("######################")
    print(f"Trait : {res['name'][0]} \n {res['explanation'][0]} \n \n")
    
    

######################
Trait : FireStimFlower 
 Fire-stimulated ï¬owering in post-ï¬re resprouters. 
 

######################
Trait : RespFire 
 Resprouting capacity after one year when most of the plant has been scorched (average proportion of
adult plants that resprout as percentage). Not reported for annual plants (which can be assumed to be
RespFire = no). 
 

######################
Trait : ChemCues 
 Germinative response to smoke (smk), ash (ash), charcoal (cha), nitrogenous compounds like KNO 3
(NC1), NaNO 2 (NC2), NH 4 Cl (NC3), NH 4 HOC 3 (NC4), NH 4 NO 3 (NC5), or response to karrikins
(KAR), cyanohydrins (CYN, including several cyanides, mandelonitrile, and glyceronitrile). The response
in indicated before the vertical bar, and the chemical cue tested, after the vertical bar (e.g. stimulation|
Smk). 
 

######################
Trait : HeatStimGerm 
 The highest intensity in heat treatments (i.e., seed exposition to dry heat > = 50 Â°C) that produce higher
germination than t

Pour des fonctions réutilisées régulièrement, j'ai décidé de créer un fichier db_request

In [15]:
from db_request import *

In [110]:
# PENSER A REMPLACER pd.unique PAR DISTINCT avec NEO4J !!

def allTaxons():
    rq = ''' match(tx:Taxon) return tx.family as family, tx.genus as genre, tx.species as espece'''
    return graph.run(rq).to_data_frame()

allTaxons=allTaxons()
allFamilies=pd.unique(allTaxons["family"])

print(f"Nombre de taxons : {len(allTaxons)}")
print(f"Nombre famille : {len(allFamilies)}")

Nombre de taxons : 2457
Nombre famille : 119


### Identification des familles dont certaines espèces sont adaptées aux incendies

Regardons dans un premier temps tous les taxons ayant au moins une modalité d'un des traits identifié précédemment. C'est à dire les taxons présentant au moins une adaptation au feu

In [109]:
fireTraits = {
    "FireStimFlower": "yes",
    "RespFire": "high",
    "ChemCues" : "stimulation",
    "HeatStimGerm" : "high|LMH",
    "SeedlEmerg" : "high",
    "SeedlSurv" : "high"
}

def allTaxonAdaptedTo(fireTraits):
    rq = ""
    maxUnion = 0
    for key, value in fireTraits.items():
        rq += f"match(tx:Taxon)<-[:TRAIT_OF_TAXON]-(d:Data {{data: '{value}'}})-[:TYPE_OF_TRAIT]->(t:Trait {{name: '{key}'}}) return tx.taxon as taxon, t.name as trait, d.data as data"
        if(maxUnion < len(fireTraits)-1):
            rq += " UNION "
            maxUnion+=1
    return graph.run(rq).to_data_frame()

allTaxonAdaptedToFire = allTaxonAdaptedTo(fireTraits)
#print(allTaxonAdaptedToFire)
print(f"{allTaxonAdaptedToFire.shape[0]} taxons présentent au moins une adaptation au feu")
print(f"soit {(allTaxonAdaptedToFire.shape[0]/len(allTaxons))*100}")

321 taxons présentent au moins une adaptation au feu


#### Focus sur le trait FireStimFlower

In [40]:
fireStimFlower = allTaxonWithTraitAndModality("FireStimFlower", "yes")
fireStimFlowerFamilies = pd.unique(fireStimFlower["famille"])
print(f"Nombre de plante possédant ce trait : {len(fireStimFlower)}")
print(f"Nombre de familles ayant au moins une espèce dont la floraison est stimulée pour les incendies : {len(fireStimFlowerFamilies)}")
print(fireStimFlowerFamilies)

Nombre de plante possédant ce trait : 24
Nombre de familles ayant au moins une espèce dont la floraison est stimulée pour les incendies : 8
['Asphodelaceae' 'Primulaceae' 'Asparagaceae' 'Amaryllidaceae' 'Liliaceae'
 'Dioscoreaceae' 'Araceae' 'Iridaceae']


In [47]:
percentageOfFireStimFlowerFamily = (len(fireStimFlowerFamilies)/len(allFamilies))*100
print(percentageOfFireStimFlowerFamily)

percentageOfFireStimFlowerTaxon = (len(fireStimFlower) / len(allTaxons)) * 100
print(percentageOfFireStimFlowerTaxon)

6.722689075630252
0.9768009768009768


Seulement 6% des familles possédent au moins une espèce ayant le trait **FireStimFlower**. 
Et moins d'1% des plantes voient leur floraison stimulée par le feu. 

D'après la documentation, ce trait indique que les plantes le possédant sont capables (voire nécessitent) de fleurir après des incendies. 
Si l'on regarde un peu plus en détail la documentation, il est indiqué que ce sont les plantes géophytes qui présentent le plus ce trait. Une plante géophyte est une plante qui possède un organe permettant de "passer la mauvaise saison" (wikipédia), tel qu'un bulbe ou bien un rhizome. On peut donc regarder la proportion de plantes géophytes dans celle présentant le trait **FireStimFlower**. 

Il existe un trait : **GrowthForm** renseignant les plantes géophytes ou non. 



In [75]:
def modalityOfOneTraitForOneTaxon(taxon, trait):
    rq = '''match(tx:Taxon {taxon: $taxon})<-[:TRAIT_OF_TAXON]-(d:Data)-[:TYPE_OF_TRAIT]->(t:Trait {name: $trait}) return tx.taxon as taxon, d.data as trait'''
    res = graph.run(rq, taxon=taxon, trait=trait).to_data_frame()
    if(len(res) != 0):
        return pd.unique(res["trait"])
    return []

def hasOneOfThisTrait(traits, taxonTraits):
    for t in taxonTraits:
        if t in traits:
            return True
    return False

In [79]:
for el in fireStimFlower["taxon"]:
    print(el, modalityOfOneTraitForOneTaxon(el, "GrowthForm"))

Asphodelus ramosus ['geophyte']
Cyclamen hederifolium ['geophyte']
Muscari racemosum ['geophyte']
Urginea maritima ['geophyte']
Narcissus triandrus ['geophyte']
Narcissus dubius ['geophyte']
Gagea peduncularis ['geophyte']
Dioscorea communis ['geophyte']
Acis nicaeensis ['geophyte']
Ornithogalum narbonense ['geophyte']
Arisarum vulgare ['geophyte']
Gladiolus illyricus ['geophyte']
Iris lutescens ['geophyte']
Lapiedra martinezii ['geophyte']
Gladiolus illyricus ['geophyte']
Romulea tempskyana ['geophyte']
Ornithogalum sphaerocarpum ['geophyte']
Allium hirtovaginatum ['geophyte']
Allium sandrasicum ['geophyte']
Allium amethystinum ['geophyte']
Gagea graeca ['geophyte']
Leopoldia comosa ['geophyte']
Asparagus stipularis ['liana']
Allium stamineum ['geophyte']


Nous pouvons constater qu'a l'exception du taxon "Asparagus stipularis", tous les taxons avec le trait **FireStimFlower** sont en fait des géophytes. 

Nous pouvons regarder combien de plantes sont géophytes dans notre base de données avec de savoir si le fait d'être géophyte suffit pour voir sa floraison stimulé par le feu ou bien s'il faut prendre en compte d'autre caractéres.

In [84]:
allGeophytes = allTaxonWithTraitAndModality("GrowthForm", "geophyte")
print(f"{round((len(fireStimFlower) / len(allGeophytes))*100)} % des geophytes possédent le trait FireStimFlower")

13 % des geophytes possédent le trait FireStimFlower


D'après le résultat précédent, nous pouvons constater que le fait d'être géophyte n'est pas suffisant pour avoir une floraison stimulée par le feu. 

#### Focus sur tous les traits lié aux incendies



#### Focus sur le trait BudSource

In [78]:
for el in fireStimFlower["taxon"]:
    print(el, modalityOfOneTraitForOneTaxon(el, "BudSource"))

Asphodelus ramosus ['storage organs']
Cyclamen hederifolium []
Muscari racemosum []
Urginea maritima ['storage organs' 'rhizomes or roots']
Narcissus triandrus []
Narcissus dubius []
Gagea peduncularis []
Dioscorea communis []
Acis nicaeensis ['storage organs']
Ornithogalum narbonense []
Arisarum vulgare ['storage organs']
Gladiolus illyricus ['storage organs']
Iris lutescens ['rhizomes']
Lapiedra martinezii ['storage organs']
Gladiolus illyricus ['storage organs']
Romulea tempskyana []
Ornithogalum sphaerocarpum []
Allium hirtovaginatum []
Allium sandrasicum []
Allium amethystinum []
Gagea graeca ['storage organs']
Leopoldia comosa ['storage organs']
Asparagus stipularis []
Allium stamineum []


Le trait **BudSource** renseigne "l'origine" des bourgeons (bud). 
On peut déjà constater que pour les plantes dont la floraison est stimulée par le feu et pour lesquelles on a l'information concernant le trait **BudSource**, seuls les organes de stockages (bulbes) et les rhizomes sont recensés. 

Nous pouvons aller plus loin dans l'étude de ces plantes présentant le trait **FireStimFlower** en regardant quel trait ils possèdent tous et voir s'il y a d'autres similarités. 